In [1]:
import os
import numpy as np
import pandas as pd
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3 import PPO,A2C
import time

In [2]:
# Choose Environment rewards type
#ENV_type = 0 -> Continues
#ENV_type = 1 -> Discrete
ENV_type =  0

# Set name of run
# TODO: change test name
#run_name = 'Continues_reward_4_V5_PPO_'
run_name =  'SOC_reward_during_episode'

In [3]:
# Load data
#path = 'G:/My Drive/לימודים/פרוייקט גמר/Data/sim data' # small dataset

path = 'C:/Users/dtubiana/PycharmProjects/chargeSystem/simulated_data'

os.chdir(path)
os.listdir(path)

df_evs_train =pd.read_excel('usr_ev_data_train.xlsx')
df_evs_test =pd.read_excel('usr_ev_data_test.xlsx')

#df_evs_train =pd.read_excel('usr_ev_data_train_max_2030.xlsx')
#df_evs_test =pd.read_excel('usr_ev_data_test_max_2030.xlsx')

df_build = pd.read_excel('Building_data.xlsx')

In [4]:
df_evs_train = df_evs_train.drop(columns=['Unnamed: 0'])
df_build = df_build.drop(columns=['Unnamed: 0'])
df_evs_test = df_evs_test.drop(columns=['Unnamed: 0'])
# create data frame to output with Initial soc

In [5]:
df_evs_out = pd.DataFrame()
df_evs_out['initial_SOC']  = pd.DataFrame(df_evs_test)['SOC'] # get initial soc to calc soc diffrance

In [6]:
#df_evs_train
#df_evs_test

In [7]:
# change directory to import EVs environment and create multi environment
os.chdir('C:/Users/dtubiana/PycharmProjects/chargeSystem')

df_evs_to_load = df_evs_train.to_dict('records')
df_build_surplus_pwr = df_build['surplus_power[kw]'].to_numpy()

N_environments = 16
if ENV_type == 0:
    "Continues environment"
    from gymnasium.envs.registration import register

    register(
        id= 'EVChargingEnv-v0',
        entry_point='EVChargingEnv_continues_reward:make_ev_charging_env',
        kwargs={'evs': df_evs_to_load, 'power_limit': df_build_surplus_pwr})


    env = make_vec_env('EVChargingEnv-v0', n_envs=N_environments, seed=42)

    env = VecNormalize(env, norm_obs=True, norm_reward=True, norm_obs_keys=['evs','power_allowed'])

elif ENV_type == 1:
    "Discrete environment"
    from gymnasium.envs.registration import register

    register(
        id= 'EVChargingEnv-v0',
        entry_point='EVChargingEnv_discrete_reward:make_ev_charging_env',
        kwargs={'evs': df_evs_to_load, 'power_limit': df_build_surplus_pwr})


    env = make_vec_env('EVChargingEnv-v0', n_envs=N_environments, seed=42)

    env = VecNormalize(env, norm_obs=True, norm_reward=True, norm_obs_keys=['evs','power_allowed'])


C:\Users\dtubiana\PycharmProjects\chargeSystem\venv\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (189, 5)
  logger.warn(


In [8]:
#env = EVChargingEnv_discrete(evs=df_evs_to_load, power_limit=df_build_surplus_pwr)
"""Function to verify environment functionality """
#from stable_baselines3.common.env_checker import check_env
#check_env(env)
"""NOT works with vectorized environments """

'NOT works with vectorized environments '

In [9]:
""" RUN algorithms """
os.chdir('C:/Users/dtubiana/PycharmProjects/chargeSystem/eval_model_V6')
models_dir = f"models/{int(time.time())}_{run_name}/"
logdir = f"logs/{int(time.time())}_{run_name}/"

from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(env, log_path='./logs/')

if not os.path.exists(models_dir):
	os.makedirs(models_dir)

if not os.path.exists(logdir):
	os.makedirs(logdir)

env.reset()

#model = PPO('MultiInputPolicy', env, verbose=2, tensorboard_log=logdir, n_steps=512)
model = A2C('MultiInputPolicy', env, verbose=2, tensorboard_log=logdir)
# save model every N steps
TIMESTEPS = 10000
iters = 0
while True:
	iters += 1
	model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"A2C", callback=eval_callback) # TODO: model name
	model.save(f"{models_dir}/{run_name}{TIMESTEPS*iters}")

Using cuda device
Logging to logs/1693061730_SOC_reward_during_episode/A2C_0
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 24        |
|    ep_rew_mean        | -1.29e+05 |
| time/                 |           |
|    fps                | 74        |
|    iterations         | 100       |
|    time_elapsed       | 107       |
|    total_timesteps    | 8000      |
| train/                |           |
|    entropy_loss       | -200      |
|    explained_variance | 0.397     |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -36.6     |
|    value_loss         | 0.0406    |
-------------------------------------
Logging to logs/1693061730_SOC_reward_during_episode/A2C_0
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 24        |
|    ep_rew_mean        | -1.25e+05 |
| time/                 |           |
|    fps                | 74


KeyboardInterrupt



tensorboard --logdir C:/Users/dtubiana/PycharmProjects/chargeSystem/eval_model_V6/logs


In [10]:
"""Load model"""
model = A2C.load('C:/Users/dtubiana/PycharmProjects/chargeSystem/eval_model_V6/models/1693055432_power_reward_unclip/power_reward_unclip240000.zip')

In [10]:
"""Create new environment for test data - Run over the last enviroment"""

df_evs_to_load = df_evs_test.to_dict('records')
df_build_surplus_pwr = df_build['surplus_power[kw]'].to_numpy()

N_environments = 16
if ENV_type == 0:
    "Continues environment"
    from gymnasium.envs.registration import register

    register(
        id= 'EVChargingEnv-v0',
        entry_point='EVChargingEnv_continues_reward:make_ev_charging_env',
        kwargs={'evs': df_evs_to_load, 'power_limit': df_build_surplus_pwr})


    env = make_vec_env('EVChargingEnv-v0', n_envs=N_environments, seed=42)

    env = VecNormalize(env, norm_obs=True, norm_reward=True, norm_obs_keys=['evs','power_allowed'])

elif ENV_type == 1:
    "Discrete environment"
    from gymnasium.envs.registration import register

    register(
        id= 'EVChargingEnv-v0',
        entry_point='EVChargingEnv_discrete_reward:make_ev_charging_env',
        kwargs={'evs': df_evs_to_load, 'power_limit': df_build_surplus_pwr})


    env = make_vec_env('EVChargingEnv-v0', n_envs=N_environments, seed=42)

    env = VecNormalize(env, norm_obs=True, norm_reward=True, norm_obs_keys=['evs','power_allowed'])


C:\Users\dtubiana\PycharmProjects\chargeSystem\venv\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment EVChargingEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
C:\Users\dtubiana\PycharmProjects\chargeSystem\venv\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (189, 5)
  logger.warn(


In [11]:
"""Run algo for predictions"""
N_steps = 24
charge_pwr = np.zeros( (N_steps,N_environments) )

# TODO: run prediction on test set to prevent over fitting
obs = env.reset()
obs = env.get_original_obs()

# Run 23 hours
for i in range(23):
    action, _states = model.predict(obs)

    obs, rewards, done, info = env.step(action)
    for env_num in range(N_environments):
        charge_pwr[i,env_num] = (info[env_num]['charge_pwr'])
    p = info[10]['charge_pwr']
    unnormalized_obs = env.get_original_obs()
    t = unnormalized_obs['time'][1]
    pa = unnormalized_obs['power_allowed'][0][t-1]
    print(f'Time: {t}, chargePWR: {p}, Power allowed {pa}')
    print()

Time: 1, chargePWR: 1252.8000000000002, Power allowed 481.7926330566406

Time: 2, chargePWR: 55.0, Power allowed 490.21636962890625

Time: 3, chargePWR: 36.7, Power allowed 494.0168762207031

Time: 4, chargePWR: 29.4, Power allowed 495.3977355957031

Time: 5, chargePWR: 22.1, Power allowed 496.5660705566406

Time: 6, chargePWR: 44.0, Power allowed 497.13995361328125

Time: 7, chargePWR: 29.4, Power allowed 470.9508972167969

Time: 8, chargePWR: 25.7, Power allowed 435.3732604980469

Time: 9, chargePWR: 47.7, Power allowed 468.2096252441406

Time: 10, chargePWR: 29.4, Power allowed 485.6842041015625

Time: 11, chargePWR: 36.7, Power allowed 487.84112548828125

Time: 12, chargePWR: 29.4, Power allowed 487.2452697753906

Time: 13, chargePWR: 33.0, Power allowed 484.8127746582031

Time: 14, chargePWR: 25.7, Power allowed 487.64337158203125

Time: 15, chargePWR: 29.4, Power allowed 489.7359924316406

Time: 16, chargePWR: 25.7, Power allowed 488.6941833496094

Time: 17, chargePWR: 29.4, Powe

In [12]:
SOC_mat = np.zeros((len(df_evs_to_load), N_environments))
for env_num in range(N_environments):
        SOC_mat[:,env_num] = (((unnormalized_obs['evs'][env_num])[:,4]).tolist())


In [13]:
total_reward =[]
for env_num in range(N_environments):
    total_reward.append(info[env_num]['total_reward'])

total_reward = np.array(total_reward)
total_reward = pd.DataFrame(total_reward, columns=['total_reward'])
tr_m = total_reward.mean()
tr_std = total_reward.std()
print(f'total_reward mean :{tr_m} ,total_reward std:  {tr_std}')

total_reward mean :total_reward   -6491.264188
dtype: float64 ,total_reward std:  total_reward    975.592709
dtype: float64


In [14]:

df_evs_out = pd.concat([df_evs_out, pd.DataFrame(unnormalized_obs['evs'][0], columns= ['Arrival_time[h]', 'TuD (int)', 'Battery capacity [KWh]', 'ENonD' ,'SOC'])], axis=1)
#df_evs_out['initial_SOC']  = pd.DataFrame(df_evs_test)['SOC'] # get initial soc to calc soc diffrance
df_evs_out

,initial_SOC,Arrival_time[h],TuD (int),Battery capacity [KWh],ENonD,SOC
0,0.456038,10.0,0.0,57.5,5.401042,0.520386
1,0.375950,22.0,0.0,77.0,8.337047,0.518807
2,0.449382,15.0,0.0,70.0,9.631797,0.449382
3,0.323799,15.0,0.0,64.0,7.331303,0.495674
4,0.416972,18.0,0.0,70.0,8.720966,0.469829
...,...,...,...,...,...,...
184,0.351860,6.0,0.0,57.5,7.976377,0.416208
185,0.414090,5.0,0.0,70.0,9.813478,0.571232
186,0.175688,1.0,0.0,70.0,8.518055,0.228545
187,0.206499,1.0,0.0,57.5,7.949520,0.270847


In [15]:
# concatenate surplus power to charge power for comparison
charge_pwr_df = pd.DataFrame(charge_pwr)
charge_pwr_df = pd.concat([charge_pwr_df,pd.Series(df_build_surplus_pwr, name= 'surplusPWR')], axis=1)
((df_evs_out['SOC'] - (df_evs_out['ENonD']/df_evs_out['Battery capacity [KWh]'] + 0.2)) < 0).sum()

48

In [16]:
"""EXPORT data to excel files"""

os.chdir('C:/Users/dtubiana/PycharmProjects/chargeSystem/eval_model_V6')

run_results_dir = f"run_results/{int(time.time())}_{run_name}/"

if not os.path.exists(run_results_dir):
	os.makedirs(run_results_dir)

os.chdir('C:/Users/dtubiana/PycharmProjects/chargeSystem/eval_model_V6/'+run_results_dir)
file_name = run_name + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')


charge_pwr_df.to_excel(writer, sheet_name='charge_pwr', index=False )
SOC_mat_df = pd.DataFrame(SOC_mat)
SOC_mat_df.to_excel(writer, sheet_name='SOC_mat', index=False)

df_evs_out.to_excel(writer, sheet_name='df_evs_out', index=False)
total_reward.to_excel(writer, sheet_name='total_reward', index=False)
# Save and close the writer
writer.close()